<a href="https://colab.research.google.com/github/JulesBendo/Test/blob/main/TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import dask.dataframe as dk
import requests
import zipfile
import os

In [24]:
# URL du fichier zip
zip_url = 'http://rhobs-public.s3-website.eu-west-3.amazonaws.com/data.zip'

In [ ]:
# Téléchargement du fichier zip
response = requests.get(zip_url)
zip_file = 'data.zip'

In [ ]:
# Ouvrir un fichier et écrire le contenu téléchargé dans ce fichier
with open(zip_file, 'wb') as f:
    f.write(response.content)

In [ ]:
# Extraire le contenu du fichier zip dans le répertoire
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
#Etant donné que le fichier se nomme "-" on le renomme en "data.csv"
os.rename('-', 'data.csv')

In [ ]:
#on supprime ce zip à présent
os.remove(zip_file)

In [ ]:
!head data.csv
#Parfait, à présent prenons connaissance des objectifs du test.

In [ ]:
#Créons notre DataFrame en sélectionnant les lignes contenants les données.
df = dk.read_csv('data.csv', delimiter=',', header=None, skiprows=9,
                 names=['a', 'b', 'c', 'd', 'e', 'f', 'g'])

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
nb_lignes = len(df)
print("Le DataFrame contient {} lignes.".format(nb_lignes))

In [ ]:
#Renomons les colonnes évidentes pour une meilleure visibilité.
df = df.rename(columns={'a': 'id', 'b': 'description', 'c': 'first_name', 'd': 'last_name', 'e': 'column1', 'f': 'column2', 'g': 'column3'})

In [ ]:
df.head(100)

In [ ]:
df.tail(100)

In [ ]:
description = df.describe().compute()
print(description)

In [ ]:
def top_3(df):
    numeric_cols = [col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'id']
    for col in numeric_cols:
      Trois_max, Trois_min = df[col].nlargest(3), df[col].nsmallest(3)
      print(f"Trois plus petites valeurs de la colonne {col} :")
      print(Trois_min.compute())
      print(f"\nTrois plus grandes valeurs de la colonne {col} :")
      print(Trois_max.compute())


In [19]:
top_3(df)

Trois plus petites valeurs de la colonne column1 :
6     0
12    0
17    0
Name: column1, dtype: int64

Trois plus grandes valeurs de la colonne column1 :
1979    5000
2910    5000
8284    5000
Name: column1, dtype: int64
Trois plus petites valeurs de la colonne column2 :
3401    -1000
12145   -1000
12503   -1000
Name: column2, dtype: int64

Trois plus grandes valeurs de la colonne column2 :
0    0
1    0
2    0
Name: column2, dtype: int64
Trois plus petites valeurs de la colonne column3 :
2552   -2470
4573   -2126
2979   -1880
Name: column3, dtype: int64

Trois plus grandes valeurs de la colonne column3 :
1017731    893775958
1018905    893775893
1017069    893775523
Name: column3, dtype: int64


In [20]:
#Maintenant que les données sont plus claires renomons les autres colonnes.
df = df.rename(columns={'e': 'deposit', 'f': 'withdrawal', 'g': 'balance'})

In [21]:
df

,id,description,first_name,last_name,column1,column2,column3
npartitions=292,,,,,,,
,int64,object,object,object,int64,int64,int64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


**MISSION 1**:

*   id(a)

    Cette colonne semble contenir des identifiants uniques pour chaque entrée dans nos données.

*   description(b)

    Cette colonne pourrait contenir des descriptions ou des informations textuelles décrivant chaque entrée dans vos données. On remarque qu'elles sont simplement notées "longue description 0-7".


*   first_name(c)

    Cette colonne semble contenir les prénoms des personnes associées à chaque entrée dans les données.

*   last_name(d)

    Cette colonne semble contenir les noms de famille des personnes  associées à chaque entrée dans les données.

*   deposit(e)

    Cette colonne présente des valeurs maximales de 5000 et des valeurs minimales de 0.
    Ceci pourrait laisser penser aux dépôts (les dépôts étant en général des ajouts de fonds) avec 5000 comme plafond et 0 comme la valeur théorique minimale du dépôt (dans le cas d'un client n'effectuant aucun dépôt).

*   withdrawal(f)

    Cette colonne présente des valeurs maximales de 0 et des valeurs minimales de -1000.
    Ceci pourrait laisser penser aux retraits(les retraits étant généralement des déductions de fonds) avec 1000 comme plafond et 0 comme la valeur théorique minimale du retrait (dans le cas d'un client n'effectuant aucun retrait).

*   balance(g)

    Cette colonne présente une valeur maximale de 893775958 et une valeur minimale de -2470.
    En analogie avec ce qui précède, ceci semble être le solde courant du compte, positif lorsque le client est en règle et négatif lorsqu'il est endetté.

In [22]:
id = [1017731, 1018905, 1017069]

# Sélectionnons les lignes correspondant aux id
selection = df[df['id'].isin(id)]

Personne = selection[['first_name', 'last_name']]

print(Personne.head())

        first_name last_name
1017069    William     Ferry
1017731   Amandine     Ferry
1018905     Céline    Ivanov


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:8110: UserWarning: Insufficient elements for `head`. 5 elements requested, only 3 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


**MISSION 2**
 Les trois personnes les plus riches sont respectivement :


1.    William Ferry

     893775958

2.   Amandine Ferry

     893775893

1.   Céline Ivanov
   
     893775523



